In [ ]:
from __future__ import division, print_function
import sys
if '..' not in sys.path:
    sys.path.append('..')

import theano
import theano.tensor as T
import lasagne

import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.metrics import confusion_matrix

In [ ]:
%matplotlib inline

# Load datasets

- the datasets are loaded/built.
- The batchsize is defined
- half of the data name (the source part) is defined

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


## Datasets Imports 

In [ ]:
from datasets.toys import make_clouds, make_circles, make_X, make_moons
from datasets.utils import make_dataset


## Clouds

In [ ]:
n_samples = 200  # Number of sample per class
n_classes = 3
batchsize = 80
_data_name = 'Clouds'
X, y = make_clouds(n_samples=n_samples, n_classes=n_classes)
source_data = make_dataset(X, y, batchsize=batchsize)

## Circles

In [ ]:
n_samples = 500  # Number of sample per class
n_classes = 5
n_dim = 2
batchsize = 60
_data_name = 'Circles'
X, y = make_circles(n_samples=n_samples, n_classes=n_classes, n_dim=n_dim)
source_data = make_dataset(X, y, batchsize=batchsize)

## X

In [ ]:
n_samples = 500  # Number of sample per class
n_classes = 5
batchsize = 60
_data_name = 'X'
X, y = make_X(n_samples=n_samples, n_classes=n_classes)
source_data = make_dataset(X, y, batchsize=batchsize)

## Moons

In [ ]:
n_samples = 500
batchsize = 60
_data_name = 'Moons'
X, y = make_moons(n_samples=n_samples)
source_data = make_dataset(X, y, batchsize=batchsize)

## MNIST

# Transform datasets

- the transformed datasets are built.
- last part of the data name (the target part) is defined

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


## Transformation Imports

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

## Data rotated

In [ ]:
data_name = _data_name+'_Rotated'
angle = 35

X_t, y_t = transform.rotate(X, y, angle=angle)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

## Data . Random Matrix

In [ ]:
data_name = _data_name+'_RMat'

X_t, y_t = transform.random_mat(X, y, normalize=False, random_state=42)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

## Data . Diag Dominant matrix

In [ ]:
data_name = _data_name+'_Diag'

X_t, y_t = transform.diag_dominant(X, y, normalize=True)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

## Data Mirror

In [ ]:
data_name = _data_name+'_Mirror'

X_t, y_t = transform.mirror(X, y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

## Data Random Permutation

In [ ]:
data_name = _data_name+'_Rperm'

X_t, y_t = transform.random_permut(X, y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

## Grid Bend

In [ ]:
data_name = _data_name+'_GridBend'
nx = 4
ny = 4
grid_noise = 0.5

X_t, y_t, grid = transform.grid_bend(X, y, nx=nx, ny=ny, noise=grid_noise)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

In [ ]:
X_src, y_src = make_clouds(n_samples=50, n_classes=6)
X_tgt, y_tgt = make_moons(n_samples=500)

# Clusters Init
Here we initialize the clusters on the *source* and on the *target* domains.

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
from sklearn.cluster import KMeans

k_src = 10
k_tgt = 11
k_means_src = KMeans(n_clusters=k_src).fit(X_src)
k_means_tgt = KMeans(n_clusters=k_tgt).fit(X_tgt)


In [ ]:
def mass(k_means):
    """
    Params
    ------
        k_means: (sklearn.cluster.KMeans instance)
    Return
    ------
        w: (numpy.array [n_clusters]) the mass of each clusters 
    """
    w = np.unique(k_means.labels_, return_counts=True)[1]
    w /= np.sum(w)
    return w
    

In [ ]:
w_src = mass(k_means_src)
# w_src = np.abs(np.sin(-w_src-np.arange(w_src.shape[0])))
# w_src /= np.sum(w_src)

w_tgt = mass(k_means_tgt)
# w_tgt = np.abs(np.sin(-w_tgt-np.arange(w_tgt.shape[0])))
# w_tgt = np.exp(-w_tgt-np.arange(w_tgt.shape[0]))
# w_tgt /= np.sum(w_tgt)
cost_mat = np.random.uniform(0,1, size=(w_src.shape[0], w_tgt.shape[0]))


In [ ]:
import visual
visual.mat(cost_mat)


# Optimal Transport Init

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
from opt_transport import opt_transp_sup, computeTransportSinkhorn, computeTransportSinkhornLabelsLpL1

In [ ]:
# transp = opt_transp_sup(k_means_src.cluster_centers_, k_means_tgt.cluster_centers_)
transp = computeTransportSinkhorn(w_src, w_tgt, cost_mat, reg=10)

In [ ]:
visual.mat(transp)

# Align

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
transp.shape, X_src.shape, k_means_src.labels_.shape

In [ ]:
from align_learn.preprocess import align


Sanity check. The cluster distribution of the aligned data should be the same as the target distrib

In [ ]:
align_idx, cluster_T = align(transp, k_means_src.labels_, k_means_tgt.labels_)
# print(np.unique(res).shape, X_src.shape, X_tgt.shape)
uniq, count = np.unique(cluster_T, return_counts=True)
# plt.plot(np.sum(transp,0), label='transp.sum(0)')
plt.plot(w_tgt, label='w_tgt')
# plt.plot(np.sum(transp,1), label='transp.sum(1)')
plt.plot(w_src, label='w_src')
plt.plot(count/cluster_T.shape[0], label='mapping')
plt.legend()
plt.show()

In [ ]:
X_S, y_S = X_src, y_src
X_T, y_T = X_tgt[align_idx], y_tgt[align_idx]

In [ ]:
visual.target_2D(X_T, y_T)
visual.target_2D(X_tgt, y_tgt)
plt.show()

In [ ]:
X_T.shape, X_S.shape

Build the probabilities to be predict

In [ ]:
def train_dataset(X_S, X_T, k_means_src, k_means_tgt, transp, align_idx, cluster_T):
    """
    """
    # Build the probabilities to be predict
    # For the source data
    proba_src = np.zeros((X_S.shape[0], k_means_src.n_clusters))
    proba_src[np.arange(X_S.shape[0]), k_means_src.labels_] = 1.
    proba_tgt = transp[k_means_src.labels_]
    Y_S = np.hstack([proba_src, proba_tgt])
    
    # Build the probabilities to be predict
    # For the aligned target data
    proba_tgt = np.zeros((X_T.shape[0], k_means_tgt.n_clusters))
    proba_tgt[np.arange(X_T.shape[0]), cluster_T] = 1.
    proba_src = transp[:, cluster_T].T
    Y_T = np.hstack([proba_src, proba_tgt])
    
    Y = np.vstack([Y_S, Y_T])
    X = np.vstack([X_S, X_T])
    return X, Y

In [ ]:
X, Y = train_dataset(X_S, X_T, k_means_src, k_means_tgt, transp, align_idx, cluster_T)

In [ ]:
X1, Y1 = train_dataset(X_S, X_T, k_means_src, k_means_tgt, transp, align_idx, cluster_T)

In [ ]:
np.all(X1==X)

In [ ]:
X.shape, Y.shape

In [ ]:
Y[0], Y[350]

# Build the Neural Network


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


## Network building
Start with the variables

In [ ]:
from nn.rgl import ReverseGradientLayer
from nn.compilers import crossentropy_sgd_mom, squared_error_sgd_mom, adversarial
from nn.training import Trainner, training


## Parameters

- Learning rates
- Hyper params
- Logger

In [ ]:
from logs import log_fname, new_logger, empty_logger

hp_lambda = 0.

# Learning rates and momentums
label_rate = 0.1
label_mom = 0.9

domain_rate = 0.1
domain_mom = 0.9

# Get a logger
logger = new_logger()


### Theano variables

In [ ]:
_shape = np.shape(source_data['X_train'])
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

# Prepare Theano variables for inputs and targets
if n_dim == 2:
    input_var = T.matrix('inputs')
    src_var = T.matrix('src')
    target_var = T.matrix('targets')
    shape = (batchsize,) + _shape[1:]
elif n_dim == 3:
    input_var = T.tensor3('inputs')
    src_var = T.tensor3('src')
    target_var = T.tensor3('targets')
    shape = (batchsize,) + _shape[1:]
elif n_dim == 4:
    input_var = T.tensor4('inputs')
    src_var = T.tensor4('src')
    target_var = T.tensor4('targets')
    shape = (batchsize,) + _shape[1:]

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


## Architecture


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape, input_var=input_var)
src_layer = lasagne.layers.InputLayer(shape=shape, input_var=src_var)
dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=25,
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
# dropout_1 = lasagne.layers.DropoutLayer(dense_1, p=0.5)
dense_2 = lasagne.layers.DenseLayer(
                dense_1,
                num_units=25,
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
feature = lasagne.layers.DenseLayer(
                input_layer,
                num_units=np.prod(shape[1:]),  # Should have same number as the input dimension
                nonlinearity=None,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
reshaper = lasagne.layers.ReshapeLayer(feature, (-1,) + shape[1:])
output_layer = reshaper

# Logs
logger.info('Building the neural network architecture for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


## Compiling 


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
# Logs
logger.info('Compiling the neural network for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Compilation
corrector_trainner = Trainner(squared_error_sgd_mom(output_layer, lr=label_rate, mom=0, 
                                                    target_var=target_var,
                                                   regularization=None, reg_param=0.1), 
                             'corrector',)

domain_trainner = Trainner(adversarial([src_layer, output_layer], hp_lambda=hp_lambda,
                                      lr=domain_rate, mom=domain_mom),
                           'domain')


## Add preprocessing for alignment


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
from align_learn.preprocess import classwise_shuffle, exhaustive_clostest, cluster_preprocess, build_clusters

# Choose preprocessing :
# corrector_trainner.preprocess = classwise_shuffle
# corrector_trainner.preprocess = exhaustive_clostest
corrector_trainner.preprocess = cluster_preprocess

model_name = ''
if corrector_trainner.preprocess is classwise_shuffle:
    model_name = 'Classwise_Corrector'
    corrector_data['labels'] = source_data['y_train']
elif corrector_trainner.preprocess is exhaustive_clostest:
    model_name = 'K-closest_Corrector'
    corrector_data['labels'] = source_data['y_train']
elif corrector_trainner.preprocess is cluster_preprocess:
    model_name = 'Cluster_Corrector'
    n_clusters = 6
    corrector_data['k'] = -1
    _y = source_data['y_train']
    classes = np.unique(_y)

    # Build the clusters for target data
    centers_array, clusters_label, centers_labels = build_clusters(corrector_data['X_train'],
                                                                   _y, n_clusters=n_clusters)
    corrector_data['X_train_centers'] = centers_array
    corrector_data['X_train_clusters'] = clusters_label
    corrector_data['centers_labels'] = centers_labels
    
    # Build the clusters for source data
    centers_array, clusters_label, centers_labels = build_clusters(corrector_data['y_train'],
                                                                   _y, n_clusters=n_clusters)
    corrector_data['y_train_centers'] = centers_array
    corrector_data['y_train_clusters'] = clusters_label

else:
    model_name = 'Pairwise_Corrector'


# Train the neural network


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


Reset the counter and the stats

In [ ]:
logger.warn('Reset the epoch counter and saved statistics')
epoch_counter = 0
final_stats = {}


## Training loop 

In [ ]:
def do_n_epoch(n_epoch):
    global epoch_counter, logger, final_stats, hp_lambda
    global corrector_data, domain_data, corrector_trainner, domain_trainner
    epoch_counter += n_epoch
    trainers = [corrector_trainner,]
    datas = [corrector_data,]
    #  If hp_lambda == 0 no need to train adversarial (faster computation)
    if hp_lambda != 0.0:
        trainers.append(domain_trainner)
        datas.append(domain_data)
    # Now do the trainning part !
    logger.info('Trainning the neural network for {} additional epochs ({} total)'.format(n_epoch, epoch_counter))
    stats = training(trainers, datas, num_epochs=n_epoch, logger=None)
    final_stats = {k: (final_stats[k]+v if k in final_stats else v) for k, v in stats.items()}

# Plot results


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


## 2D Data plot

In [ ]:
import visual
def grid_view(X, y, trainer):
    """
    Plot the corrected grid
    """
    nx = ny = 20
    x_min, x_max = np.min(X[:, 0])*2, np.max(X[:, 0])*2
    y_min, y_max = np.min(X[:, 1])*2, np.max(X[:, 1])*2

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, num=nx),
                         np.linspace(y_min, y_max, num=ny))
    X_grid = np.vstack([xx.ravel(), yy.ravel()]).T
    y_grid = np.hstack([np.ones(nx)*i for i in range(ny)])

#    Plot original grid
    ax1.plot(xx, yy, '--')
    ax1.plot(xx.T, yy.T, '--')
    ax2.plot(xx, yy, '--')
    ax2.plot(xx.T, yy.T, '--')
    
#     Transform:
#     X_tgt, y_tgt = transform.rotate(X_grid, y_grid, angle=angle)
    X_tgt, y_tgt = transform.random_mat(X_grid, y_grid, random_state=42)
#     X_tgt, y_tgt, _ = transform.grid_bend(X_grid, y_grid, nx=nx, ny=ny, grid=grid)
#     X_tgt, y_tgt = TT(X_grid, y_grid)
    
    X_corr = np.array(trainer.output(X_tgt)).reshape((-1, 2))
    xx = X_corr[:, 0].reshape(ny, nx)
    yy = X_corr[:, 1].reshape(ny, nx)
    

    # Plot data test points (source + corrected) on left fig
    visual.corrected_2D(X, y, ax=ax1)
    ax1.plot(xx, yy)
    ax1.plot(xx.T, yy.T)

    xx = X_tgt[:, 0].reshape(ny, nx)
    yy = X_tgt[:, 1].reshape(ny, nx)
    visual.corrected_2D(X, y, ax=ax2)
    ax2.plot(xx, yy)
    ax2.plot(xx.T, yy.T)
    return fig, (ax1, ax2)
    

In [ ]:
fig_dir = '/home/victor/Workspace/Stage/DANN/fig/'
fig_title = fig_dir+data_name+model_name
from datasets.utils import Dataset

# Play !


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Clusters init](#Clusters-Init)
- [4. Optimal transport init](#Optimal-Transport-Init)
- [5. Align](#Align)
- [6. Init of the NN](#Build-the-Neural-Network)
- [7. Architecture](#Architecture)
- [8. Compiling](#Compiling)
- [9. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [10. Playground](#Play-!)


In [ ]:
import visual

In [ ]:
print(model_name, data_name)
do_n_epoch(50)

# ================
# Compute the correction on test data
# ================
corrected_data = Dataset(
    X_test=np.array(corrector_trainner.output(corrector_data['X_test'])).reshape((-1, 2)),
    y_test=source_data.y_test
)


# ================
# Data visualisation
# ================
if not n_dim > 2:
    # Init figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # Plot data test points (source + target) on right fig
    visual.target_2D(source_data.X_test, source_data.y_test, ax=ax1);
    visual.source_2D(target_data.X_test, target_data.y_test, ax=ax1);
    visual.add_legend(ax1, title=data_name+model_name)
    
    # Plot data test points (source + corrected) on left fig
    visual.target_2D(source_data.X_test, source_data.y_test, ax=ax2);
    visual.corrected_2D(corrected_data.X_test, source_data.y_test, ax=ax2);
    visual.add_legend(ax2, title=data_name+model_name)
    
    # Plot cluster centers and cluster mapping 
#     if 'preprocess' in corrector_data and 'X_train_centers' in corrector_data:
#         corrected_data['X_train_centers'] = np.array(
#             corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         idx = corrector_data['preprocess']
#         X = np.array(corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         Y = corrector_data['y_train_centers']
#         Y = Y[idx]
#         visual.centers_source(Y, ax=ax2)
#         visual.centers_corrected(X, ax=ax2)
#         visual.mapping(X, Y, ax=ax2)
#         visual.centers_source(Y, ax=ax1)
#         visual.centers_target(corrector_data['X_train_centers'], ax=ax1)
#     SAVE
    fig.savefig(fig_title+'-DATA.png')
    fig.show()

# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(final_stats, regex='corrector .* loss')
#     SAVE
fig.tight_layout()
fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
visual.learning_curve(final_stats, regex='domain.* acc');

# ================
# Grid check
# ================
# fig, (ax1, ax2) = grid_view(corrected_data.X_test, corrected_data.y_test, corrector_trainner)
# fig.savefig(fig_title+'-GridCheck.png')
# fig.show()

# ================
# Image samples
# ================
if data_name.startswith('MNIST'):
    visual.img_samples([source_data, target_data, 
                     {'X_test': np.array(corrector_trainner.output(
                    corrector_data['X_test'])).reshape((-1,)+corrector_data['X_test'].shape[1:])}])

# ================
# Weights visualisation
# ================
layers = lasagne.layers.get_all_layers(feature)
fig, axes = plt.subplots(len(layers)//2, 2, figsize=(20, 6))
axes = axes.ravel()
for i, l in enumerate(layers[1:]):
    if hasattr(l, 'W'):
        visual.mat(l.W.get_value(), ax = axes[i])
        visual.add_legend(axes[i], title='Layer {} Weights'.format(i))
#     SAVE
fig.savefig(fig_title+'-W.png')
fig.show()

# ================
# Sanity check
# ================
print('Loss : Identitée / Correction Finale')
print(np.mean((source_data['X_test']-target_data['X_test'])**2))
print(np.mean((source_data['X_test']-corrected_data['X_test'])**2))

In [ ]:
'Done'

In [ ]:
corrector_data.batchsize, corrector_data.X_val.shape

In [ ]:
fig, ax = plt.subplots()
corrected_data['X_train_centers'] = np.array(
    corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
idx = corrector_data['preprocess']
X = np.array(corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
Y = corrector_data['y_train_centers']
Y = Y[idx]
visual.target_2D(source_data.X_test, source_data.y_test, ax=ax);
visual.corrected_2D(corrected_data.X_test, source_data.y_test, ax=ax);
visual.centers_source(Y, ax=ax)
visual.centers_corrected(X, ax=ax)
visual.mapping(X, Y, ax=ax)
visual.add_legend(ax, title=data_name+model_name)
fig.savefig(fig_dir+'C-Moons-150'+'.png',bbox_inches='tight')
fig.show()


### TSNE